<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Topic Modeling
## *Data Science Unit 4 Sprint 1 Assignment 4*

Analyze a corpus of Amazon reviews from Unit 4 Sprint 1 Module 1's lecture using topic modeling: 

- Fit a Gensim LDA topic model on Amazon Reviews
- Select appropriate number of topics
- Create some dope visualization of the topics
- Write a few bullets on your findings in markdown at the end
- **Note**: You don't *have* to use generators for this assignment

In [1]:
import numpy as np
import gensim
import os
import re

from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim import corpora

from gensim.models.ldamulticore import LdaMulticore

import pandas as pd
import spacy
from spacy.lang.en import English
nlp = spacy.load('en_core_web_lg')

C:\Users\Samue\Anaconda3\envs\DS-U4-S1-NLP\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
data = pd.read_csv('./data/Datafiniti_Amazon_Consumer_Reviews/Amazon_Consumer_Reviews.csv')

In [3]:
data.head()

,id,dateAdded,dateUpdated,name,asins,brand,categories,primaryCategories,imageURLs,keys,...,reviews.didPurchase,reviews.doRecommend,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.username,sourceURLs
0,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,3,https://www.amazon.com/product-reviews/B00QWO9...,I order 3 of them and one of the item is bad q...,... 3 of them and one of the item is bad quali...,Byger yang,"https://www.barcodable.com/upc/841710106442,ht..."
1,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,4,https://www.amazon.com/product-reviews/B00QWO9...,Bulk is always the less expensive way to go fo...,... always the less expensive way to go for pr...,ByMG,"https://www.barcodable.com/upc/841710106442,ht..."
2,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,Well they are not Duracell but for the price i...,... are not Duracell but for the price i am ha...,BySharon Lambert,"https://www.barcodable.com/upc/841710106442,ht..."
3,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,Seem to work as well as name brand batteries a...,... as well as name brand batteries at a much ...,Bymark sexson,"https://www.barcodable.com/upc/841710106442,ht..."
4,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,These batteries are very long lasting the pric...,... batteries are very long lasting the price ...,Bylinda,"https://www.barcodable.com/upc/841710106442,ht..."


In [10]:
STOPWORDS = set(STOPWORDS).union(set(['amazon']))

def tokenize(text):
    return [token for token in simple_preprocess(text) if token not in STOPWORDS]

In [11]:
data['tokens'] = data['reviews.text'].apply(tokenize)

In [13]:
data['tokens'][0:10]

0    [order, item, bad, quality, missing, backup, s...
1               [bulk, expensive, way, products, like]
2                             [duracell, price, happy]
3              [work, brand, batteries, better, price]
4             [batteries, long, lasting, price, great]
5    [bought, lot, batteries, christmas, amazonbasi...
6    [ive, problame, batteries, ordered, past, plea...
7    [looking, cheap, non, rechargeable, batteries,...
8    [hold, high, power, juice, like, energizer, du...
9    [amazonbasics, aa, aaa, batteries, appear, goo...
Name: tokens, dtype: object

In [14]:
id2word = corpora.Dictionary(data['tokens'])

In [15]:
corpus = [id2word.doc2bow(text) for text in data['tokens']]

In [16]:
lda = LdaMulticore(corpus=corpus,
                  id2word=id2word,
                  random_state=723812,
                  num_topics=15,
                  passes=10,
                  workers=8
                  )

In [17]:
lda.print_topics()

[(0,
  '0.063*"tablet" + 0.023*"got" + 0.022*"case" + 0.019*"kids" + 0.019*"great" + 0.019*"use" + 0.017*"love" + 0.014*"loves" + 0.013*"year" + 0.012*"perfect"'),
 (1,
  '0.055*"kindle" + 0.020*"new" + 0.020*"tablet" + 0.016*"books" + 0.013*"use" + 0.012*"time" + 0.012*"games" + 0.010*"love" + 0.010*"bought" + 0.010*"great"'),
 (2,
  '0.023*"store" + 0.019*"batteries" + 0.015*"google" + 0.015*"play" + 0.013*"tablet" + 0.011*"use" + 0.009*"like" + 0.008*"apps" + 0.008*"price" + 0.008*"cheap"'),
 (3,
  '0.054*"kindle" + 0.026*"use" + 0.026*"books" + 0.026*"reading" + 0.024*"love" + 0.022*"read" + 0.017*"size" + 0.012*"product" + 0.011*"great" + 0.011*"perfect"'),
 (4,
  '0.204*"good" + 0.056*"price" + 0.034*"quality" + 0.028*"product" + 0.021*"batteries" + 0.014*"far" + 0.014*"better" + 0.013*"value" + 0.013*"battery" + 0.012*"like"'),
 (5,
  '0.087*"easy" + 0.060*"use" + 0.039*"love" + 0.033*"kids" + 0.028*"tablet" + 0.022*"great" + 0.014*"nice" + 0.012*"games" + 0.011*"product" + 0.01

In [21]:
words = [re.findall(r'"([^"]*)"', t[1]) for t in lda.print_topics()]

In [22]:
topics = [' '.join(t[0:5]) for t in words]

In [23]:
for id, t in enumerate(topics):
    print(f"------ Topic {id} ------")
    print(t, end="\n")
    print("\n")

------ Topic 0 ------
tablet got case kids great


------ Topic 1 ------
kindle new tablet books use


------ Topic 2 ------
store batteries google play tablet


------ Topic 3 ------
kindle use books reading love


------ Topic 4 ------
good price quality product batteries


------ Topic 5 ------
easy use love kids tablet


------ Topic 6 ------
batteries long great price work


------ Topic 7 ------
great price tablet works product


------ Topic 8 ------
screen kindle device battery like


------ Topic 9 ------
loves bought old year tablet


------ Topic 10 ------
batteries use battery like bought


------ Topic 11 ------
tablet apps kids great games


------ Topic 12 ------
tablet great good apps friendly


------ Topic 13 ------
far batteries battery need expected


------ Topic 14 ------
tablet tablets bought best love




In [25]:
import pyLDAvis.gensim

pyLDAvis.enable_notebook()

In [28]:
pyLDAvis.gensim.prepare(lda, corpus, id2word)

C:\Users\Samue\Anaconda3\envs\DS-U4-S1-NLP\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6      0.258307 -0.047604       1        1  13.420033
11    -0.099537 -0.024442       2        1   9.085878
9     -0.183918  0.032150       3        1   8.919003
8     -0.003702  0.063678       4        1   8.685795
10     0.158215  0.115888       5        1   7.597081
7     -0.003538 -0.215908       6        1   7.077783
5     -0.148810 -0.025693       7        1   6.600888
3     -0.073905  0.003573       8        1   6.290259
1     -0.084984  0.082427       9        1   5.295617
4      0.185699 -0.135618      10        1   4.871618
13     0.135722  0.118859      11        1   4.758113
2      0.080060  0.079501      12        1   4.668763
14    -0.028923  0.015848      13        1   4.328267
0     -0.118160  0.022785      14        1   4.313531
12    -0.072528 -0.085446      15        1   4.087380, topic_info=     Category         Freq           Term        Total  loglift  logprob
34    Default  5526.000000           good  5526.000000  30.0000  30.0000
19    Default  7401.000000      batteries  7401.000000  29.0000  29.0000
22    Default  8893.000000          great  8893.000000  28.0000  28.0000
4196  Default  6867.000000         tablet  6867.000000  27.0000  27.0000
3156  Default  2977.000000         kindle  2977.000000  26.0000  26.0000
901   Default  2794.000000           easy  2794.000000  25.0000  25.0000
2615  Default  2219.000000          loves  2219.000000  24.0000  24.0000
18    Default  4873.000000          price  4873.000000  23.0000  23.0000
24    Default  2440.000000           long  2440.000000  22.0000  22.0000
96    Default  4349.000000            use  4349.000000  21.0000  21.0000
604   Default  2162.000000            old  2162.000000  20.0000  20.0000
107   Default  1799.000000           year  1799.000000  19.0000  19.0000
201   Default  2450.000000           kids  2450.000000  18.0000  18.0000
8     Default  1287.000000        quality  1287.000000  17.0000  17.0000
28    Default  3190.000000         bought  3190.000000  16.0000  16.0000
92    Default  3420.000000           love  3420.000000  15.0000  15.0000
3172  Default  1480.000000          books  1480.000000  14.0000  14.0000
98    Default  1434.000000          works  1434.000000  13.0000  13.0000
1614  Default   963.000000       daughter   963.000000  12.0000  12.0000
21    Default  1258.000000          brand  1258.000000  11.0000  11.0000
204   Default  1197.000000            far  1197.000000  10.0000  10.0000
771   Default  1716.000000          games  1716.000000   9.0000   9.0000
445   Default   991.000000          value   991.000000   8.0000   8.0000
2049  Default  1827.000000           apps  1827.000000   7.0000   7.0000
173   Default  1996.000000        product  1996.000000   6.0000   6.0000
75    Default   855.000000         brands   855.000000   5.0000   5.0000
1058  Default  1081.000000           gift  1081.000000   4.0000   4.0000
10    Default  2201.000000           work  2201.000000   3.0000   3.0000
2244  Default  1596.000000         screen  1596.000000   2.0000   2.0000
31    Default   901.000000      christmas   901.000000   1.0000   1.0000
...       ...          ...            ...          ...      ...      ...
3101  Topic15    32.339005  functionality    68.373055   2.4486  -5.9876
451   Topic15   106.939857        storage   381.189636   1.9262  -4.7916
1952  Topic15    64.596336            add   201.188141   2.0612  -5.2957
3650  Topic15    23.716673           slot    49.889183   2.4536  -6.2977
681   Topic15    98.555511         camera   398.540375   1.8001  -4.8733
1766  Topic15    20.432100         credit    40.590279   2.5108  -6.4468
4279  Topic15    19.674257           rear    38.351448   2.5298  -6.4846
2049  Topic15   240.250183           apps  1827.855225   1.1680  -3.9822
4196  Topic15   528.735718         tablet  6867.816406   0.6332  -3.1934
664   Topic15    45.473515       pictures   15

## Markdown Notes
### 1. In topic 7, there's enough people spelling 'long' as 'looooooong' that it's actually tied with 'audibles'.
### 2. There's a few instances of mispellings being recorded which means that a lot of people were mispelling the word the same way.
### 3. There's a lot reviews about batteries and/or tablets but there are no reviews about the Amazon Alexa range of products which
### could be caused by the age of the buyers???

## Stretch Goals

* Incorporate Named Entity Recognition in your analysis
* Incorporate some custom pre-processing from our previous lessons (like spacy lemmatization)
* Analyze a dataset of interest to you with topic modeling